In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy

from numpy import random
from scipy.stats import uniform
from scipy.sparse import linalg, random

In [ ]:
# Parameters
m = n = 100
rv_magnitude = 1
dens = 1.0
lower = np.zeros(n)
upper = np.ones(n)
EPS = 0.01
num_trials = 20

In [ ]:
def evaluate_with_replacement(f, i, x, z):
    "Evaluates f(x_{-i}, z)"
    xi = x[i]
    x[i] = z
    val = f(x)
    x[i] = xi
    return val

In [ ]:
def diff(f, i, x):
    "Approximately computes the ith derivative of f at x."
    old_value = f(x)
    new_value = evaluate_with_replacement(f, i, x, x[i] + EPS)
    
    return (new_value - old_value) / EPS

In [ ]:
def optimize(f, i, x, low, high):
    "Approximately computes argmax_{z} f(x_{-i}, z)."
    best_value = f(x)
    best_xi = x[i]
    
    for z in np.arange(low, high+EPS, EPS):
        curr_value = evaluate_with_replacement(f, i, x, z)
        if curr_value > best_value:
            best_value = curr_value
            best_xi = z
    
    return best_xi

In [ ]:
def generate_quadratic_test_function():
    "Generates a function f of the form (1/2 x^T H x + h^T x + c)."
    # Decide H
    rvs = scipy.stats.uniform(loc=-rv_magnitude, scale=rv_magnitude).rvs
    H = scipy.sparse.random(m, n, density=dens, data_rvs = rvs)
    H = 0.5 * H + 0.5 * H.transpose() # Symmetrize H
    # Decide h
    h = np.random.uniform(0.0, 1.0, n)
    # Pick the right constant c so that f(lower) + f(upper) = 0
    c = -0.5 * (0.5 * np.dot(upper, H.dot(upper)) + np.dot(h, upper) +
                0.5 * np.dot(lower, H.dot(lower)) + np.dot(h, lower))
    def f(x):
        "0.5 x^T H x - h^t x + c"
        return 0.5 * np.dot(x, H.dot(x)) + np.dot(h, x) + c
    return (f, H, h, c)

In [ ]:
def generate_weak_quadratic_test_function():
    "Generates a function f of the form (1/2 x^T H x + h^T x + c)."
    # Decide H
    rvs = scipy.stats.uniform(loc=-rv_magnitude, scale=rv_magnitude).rvs
    H = scipy.sparse.random(m, n, density=dens, data_rvs = rvs)
    H = 0.5 * H + 0.5 * H.transpose() # Symmetrize H
    # Make the diagonal entries of H positive
    for i in range(n):
        H[i,i] = -H[i,i]
    # Decide h
    h = np.random.uniform(0.0, 1.0, n)
    # Pick the right constant c so that f(lower) + f(upper) = 0
    c = -0.5 * (0.5 * np.dot(upper, H.dot(upper)) + np.dot(h, upper) +
                0.5 * np.dot(lower, H.dot(lower)) + np.dot(h, lower))
    def f(x):
        "0.5 x^T H x - h^t x + c"
        return 0.5 * np.dot(x, H.dot(x)) + np.dot(h, x) + c
    return (f, H, h, c)

In [ ]:
def generate_softmax_test_function():
    eigens = np.exp(np.random.uniform(-0.5, 1.0, n))
    D = scipy.sparse.diags(eigens, 0)
    V = scipy.linalg.orth(np.random.uniform(0.0, 1.0, (n, n)))
    L = V.dot(D.dot(V.transpose()))
    I = np.identity(n)
    def f(x):
        return np.log(scipy.linalg.det(scipy.sparse.diags(x).dot(L - I) + I))
    return (f, L, D, V)

In [ ]:
def BMBK(f, x, y, i):
    "Returns the new value for x_i and y_i according to BMBK"
    ux = x[i]
    uy = y[i]
    ua = optimize(f, i, x, ux, uy)
    ub = optimize(f, i, y, ux, uy)
    
    a1 = f(x)
    x[i] = ua; a2 = f(x); x[i] = ux
    
    b1 = f(y)
    y[i] = ub; b2 = f(y); y[i] = uy
    
    if a2-a1 >= b2-b1:
        return ua
    else:
        return ub

In [ ]:
def BINARY(f, x, y, i):
    "Returns the new value for x_i and y_i according to our binary search criterion."
    low = ux = x[i]
    high = uy = y[i]
    
    while high - low > EPS:
        mid = (low + high) / 2.0
        
        x[i] = y[i] = mid
        xslope = diff(f, i, x) # decreasing in m
        yslope = -diff(f, i, y) # increasing in m
        
        if (mid - ux) * yslope <= (uy - mid) * xslope: # LHS is increasing in m, RHS is decreasing in m
            low = mid
        else:
            high = mid
    x[i] = ux
    y[i] = uy
    return low

In [ ]:
def concave_envelope(g, h, low, high):
    st = []
    # Trace out a (g, h) curve from (0, beta) to (alpha, 0)
    for curr in np.arange(low, high + 0.5 * EPS, EPS):
        st.append((g(curr), h(curr), curr))
        # If a point isn't to the right of the previous one, then weak-DR implies that it is below.
        if len(st) >= 2 and st[-1][0] <= st[-2][0]:
            st.pop()
        while len(st) >= 3 and (st[-1][1] - st[-2][1]) * (st[-2][0] - st[-3][0]) >= (st[-2][1] - st[-3][1]) * (st[-1][0] - st[-2][0]):
            st.pop(-2)
    return st

def GAME(f, x, y, i):
    "Returns the new value for x_i and y_i according to our game criterion."
    ux = x[i]
    uy = y[i]
    zl = optimize(f, i, y, ux, uy)
    zu = optimize(f, i, x, ux, uy)
    
    if zu <= zl:
        return zl # doesn't matter which between ua or ub
                  
    def g(z):
        return evaluate_with_replacement(f, i, x, z) - evaluate_with_replacement(f, i, x, zl)
    def h(z):
        return evaluate_with_replacement(f, i, y, z) - evaluate_with_replacement(f, i, y, zu)
    
    alpha = g(zu)
    beta = h(zl)
    
    envelope = concave_envelope(g, h, zl, zu)
    
    def line_distance(p):
        return (p[0] - alpha) - (p[1] - beta)
    
    for i in range(len(envelope) - 1):
        d1 = line_distance(envelope[i])
        d2 = line_distance(envelope[i+1])
        if (d1 <= 0) and (d2 >= 0):
            lmda = d2 / (d2 - d1)
            if np.random.binomial(1, lmda) == 1:
                return envelope[i][2]
            else:
                return envelope[i+1][2]
    

In [ ]:
def DoubleGreedyFramework(f, lower, upper, selection_alg):
    "Runs the BFNS framework for submodular maximization."
    perm = np.random.permutation(n)
    x = np.copy(lower)
    y = np.copy(upper)
    for i in perm:
        # Make x and y agree on coordinate i
        x[i] = y[i] = selection_alg(f, x, y, i)
    return x

In [ ]:
# Experiment 1
vals_BMBK = []
vals_GAME = []
vals_BINARY = []
functions = []
for x in range(num_trials):
    (f, H, h, c) = generate_quadratic_test_function()
    functions.append(f)
    
    sln_BMBK = DoubleGreedyFramework(f, lower, upper, BMBK)
    sln_GAME = DoubleGreedyFramework(f, lower, upper, GAME)
    sln_BINARY = DoubleGreedyFramework(f, lower, upper, BINARY)
    
    val_BMBK = f(sln_BMBK)
    val_GAME = f(sln_GAME)
    val_BINARY = f(sln_BINARY)
    
    vals_BMBK.append(val_BMBK)
    vals_GAME.append(val_GAME)
    vals_BINARY.append(val_BINARY)

In [ ]:
print("BMBK: %f" % np.mean(vals_BMBK))
print("GAME: %f" % np.mean(vals_GAME))
print("BINARY: %f" % np.mean(vals_BINARY))

In [ ]:
fig = plt.figure()
plt.boxplot([vals_BMBK, vals_GAME, vals_BINARY], labels=['BMBK', 'Game Strategy', 'Binary Search'])
# plt.savefig('QPstrong.png', dpi=fig.dpi)

In [ ]:
# Experiment 2
vals_BMBK = []
vals_GAME = []
vals_BINARY = []
functions = []
for x in range(num_trials):
    (f, H, h, c) = generate_weak_quadratic_test_function()
    functions.append(f)
    
    sln_BMBK = DoubleGreedyFramework(f, lower, upper, BMBK)
    sln_GAME = DoubleGreedyFramework(f, lower, upper, GAME)
    sln_BINARY = DoubleGreedyFramework(f, lower, upper, BINARY)
    
    val_BMBK = f(sln_BMBK)
    val_GAME = f(sln_GAME)
    val_BINARY = f(sln_BINARY)
    
    vals_BMBK.append(val_BMBK)
    vals_GAME.append(val_GAME)
    vals_BINARY.append(val_BINARY)

In [ ]:
print("BMBK: %f" % np.mean(vals_BMBK))
print("GAME: %f" % np.mean(vals_GAME))
print("BINARY: %f" % np.mean(vals_BINARY))

In [ ]:
fig = plt.figure()
plt.boxplot([vals_BMBK, vals_GAME, vals_BINARY], labels=['BMBK', 'Game Strategy', 'Binary Search'])
# plt.savefig('QPweak.png', dpi=fig.dpi)

In [ ]:
# Experiment 3
vals_BMBK = []
vals_GAME = []
vals_BINARY = []
functions = []
for x in range(num_trials):
    (f, L, D, V) = generate_softmax_test_function()
    functions.append(f)
    
    sln_BMBK = DoubleGreedyFramework(f, lower, upper, BMBK)
    sln_GAME = DoubleGreedyFramework(f, lower, upper, GAME)
    sln_BINARY = DoubleGreedyFramework(f, lower, upper, BINARY)
    
    val_BMBK = f(sln_BMBK)
    val_GAME = f(sln_GAME)
    val_BINARY = f(sln_BINARY)
    
    vals_BMBK.append(val_BMBK)
    vals_GAME.append(val_GAME)
    vals_BINARY.append(val_BINARY)

In [ ]:
print("BMBK: %f" % np.mean(vals_BMBK))
print("GAME: %f" % np.mean(vals_GAME))
print("BINARY: %f" % np.mean(vals_BINARY))

In [ ]:
fig = plt.figure()
plt.boxplot([vals_BMBK, vals_GAME, vals_BINARY], labels=['BMBK', 'Game Strategy', 'Binary Search'])
# plt.savefig('softmax.png', dpi=fig.dpi)